## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-20-06-58-00 +0000,wsj,Henkel in Talks Over Potential Acquisition of ...,https://www.wsj.com/business/deals/henkel-in-t...
1,2026-01-20-06-43-05 +0000,nypost,Surfer bitten in 4th shark attack off Australi...,https://nypost.com/2026/01/20/world-news/surfe...
2,2026-01-20-06-39-12 +0000,wapo,Indiana judge and his wife injured in shooting...,https://www.washingtonpost.com/nation/2026/01/...
3,2026-01-20-06-39-00 +0000,wsj,President Trump has ramped up pressure on Euro...,https://www.wsj.com/politics/elections/why-an-...
4,2026-01-20-06-35-15 +0000,bbc,'Emotional and financial' pressure of funeral ...,https://www.bbc.com/news/articles/c0kep1kvn8ko...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2342/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
26,trump,71
34,greenland,30
221,new,14
91,spain,12
239,ice,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
9,2026-01-20-06-08-01 +0000,bbc,Trump says he will '100%' carry out Greenland ...,https://www.bbc.com/news/articles/c4g5345ylk0o...,151
79,2026-01-20-00-00-23 +0000,bbc,"Greenland 'will stay Greenland', former Trump ...",https://www.bbc.com/news/articles/c98p6m0r53no...,145
243,2026-01-19-12-12-55 +0000,nypost,Trump says he no longer needs to ‘think purely...,https://nypost.com/2026/01/19/us-news/trump-sa...,141
3,2026-01-20-06-39-00 +0000,wsj,President Trump has ramped up pressure on Euro...,https://www.wsj.com/politics/elections/why-an-...,138
242,2026-01-19-12-13-28 +0000,cbc,Trump ties his threat to take Greenland to bei...,https://www.cbc.ca/news/world/greenland-europe...,134


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
9,151,2026-01-20-06-08-01 +0000,bbc,Trump says he will '100%' carry out Greenland ...,https://www.bbc.com/news/articles/c4g5345ylk0o...
156,58,2026-01-19-19-24-57 +0000,nyt,High-Speed Train Crash in Spain Leaves at Leas...,https://www.nytimes.com/live/2026/01/18/world/...
130,49,2026-01-19-21-28-42 +0000,startribune,The latest: Department of Justice plans to app...,https://www.startribune.com/ice-raids-minnesot...
233,41,2026-01-19-13-00-00 +0000,wsj,Half of U.S. voters say the Trump administrati...,https://www.wsj.com/politics/policy/voters-ske...
230,39,2026-01-19-13-17-00 +0000,wsj,European Stock Markets Tumble on Trump’s Tarif...,https://www.wsj.com/finance/stocks/global-equi...
218,35,2026-01-19-14-22-01 +0000,nypost,Haunting image shows pile of abandoned shoes a...,https://nypost.com/2026/01/19/world-news/haunt...
102,35,2026-01-19-23-17-35 +0000,nypost,Massive $6B offshore Long Island wind project ...,https://nypost.com/2026/01/19/us-news/massive-...
90,34,2026-01-19-23-46-47 +0000,nypost,Kamala Harris aides grilled Tim Walz on China ...,https://nypost.com/2026/01/19/us-news/kamala-h...
109,32,2026-01-19-22-58-10 +0000,nypost,"LI school board boss, who launched fight to sa...",https://nypost.com/2026/01/19/us-news/li-schoo...
252,30,2026-01-19-11-41-57 +0000,bbc,Ecstasy and now a day off - Senegal fans aroun...,https://www.bbc.com/news/articles/ce3ekz4yx21o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
